<a href="https://www.kaggle.com/code/mafiosoquasar/sales-forecasting-with-pytorch?scriptVersionId=209647961" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import argparse
import os
import random,numpy,pandas
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'

In [2]:
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import torchvision.transforms.functional as RF
from torchvision.models.feature_extraction import create_feature_extractor
from PIL import Image
import numpy as np
import random,cv2,pandas,os,numpy
import matplotlib.pyplot as plt
import matplotlib.animation as animation

In [3]:
seed = 999
print("Random Seed: ", seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

nz = 100
beta1 = 0.5
lr = 0.0001
batch_size=100
ngpu=1
ngf,nc = 3,3
ndf = 64

device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

Random Seed:  999


In [4]:
x = pandas.read_csv("/kaggle/input/rohlik-sales-forecasting-challenge-v2/sales_train.csv").sort_values(by=['date', 'warehouse'])

x = torch.from_numpy(numpy.nan_to_num(x[['total_orders', 'sell_price_main', 'type_0_discount', 'type_1_discount',
                                         'type_2_discount', 'type_3_discount', 'type_4_discount',  'type_5_discount', 
                                         'type_6_discount']].to_numpy().astype(numpy.float32)).reshape((13, 308263, 9)))
x.shape #4315682

class EC_NET(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.rafire = nn.Sequential(       
            nn.Linear(9, 1524),
            nn.Linear(1524, 824),
            nn.Linear(824, 424),
            nn.Linear(424, 124),
            nn.Linear(124, 300)
        )

    def forward(self,x):
        
        return self.rafire(x)
ec_net = EC_NET().type(torch.float32).to(device).eval()
ec_net= nn.DataParallel(ec_net).to(device)
#ec_net.load_state_dict(torch.load("/kaggle/input/encoder-weight-data/weight.pth",weights_only=False,map_location=torch.device('cpu')))

j=0
for i in x:
    print(f"loding batch : {j}")
    if j==0:
        encode = ec_net(i).cpu().detach().numpy().reshape((len(i),10,10,3))
        print(encode.shape)
    else:
        encode = numpy.concatenate((encode, ec_net(i).cpu().detach().numpy().reshape((len(i),10,10,3))), axis=0)
    #if j==25:
    #    break
    j+=1

x=torch.Tensor(encode)[::100]

loding batch : 0


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):


(308263, 10, 10, 3)
loding batch : 1


/opt/conda/lib/python3.10/site-packages/torch/nn/modules/linear.py:117: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at /usr/local/src/pytorch/aten/src/ATen/cuda/CublasHandlePool.cpp:135.)
  return F.linear(input, self.weight, self.bias)


loding batch : 2
loding batch : 3
loding batch : 4
loding batch : 5
loding batch : 6
loding batch : 7
loding batch : 8
loding batch : 9
loding batch : 10
loding batch : 11
loding batch : 12


In [5]:
y=pandas.read_csv("/kaggle/input/rohlik-sales-forecasting-challenge-v2/sales_train.csv").sort_values(by=['date', 'warehouse'])
y=torch.from_numpy(numpy.nan_to_num(y['sales'].to_numpy().astype(numpy.float32))).reshape(len(y),1)[::100]

In [6]:
x.shape, y.shape

(torch.Size([40075, 10, 10, 3]), torch.Size([40075, 1]))

In [7]:
train_x = torch.utils.data.DataLoader(x,batch_size=batch_size)
train_y = torch.utils.data.DataLoader(y,batch_size=batch_size)

In [8]:
class EffnetModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        effnet = torchvision.models.efficientnet_v2_m(weights=torchvision.models.efficientnet.EfficientNet_V2_M_Weights.DEFAULT)
        self.model = create_feature_extractor(effnet, ['flatten'])
        self.nn_fracture = torch.nn.Sequential(
            torch.nn.Linear(1280, 2),
        )
    def forward(self, x):
        x = self.model(x)['flatten']
        return self.nn_fracture(x)

In [9]:
EFF_NET = EffnetModel().float()
EFF_NET= nn.DataParallel(EFF_NET).to(device)

Downloading: "https://download.pytorch.org/models/efficientnet_v2_m-dc08266a.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_v2_m-dc08266a.pth
100%|██████████| 208M/208M [00:01<00:00, 186MB/s]


In [10]:
criterion = nn.L1Loss()

optimizer = optim.AdamW(EFF_NET.parameters(), lr=lr,betas=(beta1, 0.999))
scheduler=torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.86)

In [11]:
high_rf,i,j_r,k,z_w_=100000,0,0,0,[]
if os.path.exists(f"/kaggle/working/jsr_rf")==False:
    os.mkdir(f"/kaggle/working/jsr_rf")
    
correct,total=0,0

while(True):
    optimizer.zero_grad()
    for data,label in zip(train_x,train_y):
        #z_ = nn.Embedding(10, 3, dtype=torch.float64)
        #print(z_(torch.FloatTensor(data)))
        
        output = EFF_NET(torch.reshape(data,(len(data),3,10,10)).to(device)).view(-1)
        err_real = criterion(output, label.to(device))
        err_real.backward()
        optimizer.step()
        #print(err_real.item())
        
    print(f"EPOCH : {i} LOSS_wl : {err_real.item()}")

    if len(z_w_)>=3:
        if len([True for i in range(1,4) if z_w_[len(z_w_)-i]<=z_w_[len(z_w_)-3] and z_w_[len(z_w_)-i]>=z_w_[len(z_w_)-4]])==3:
            z_w_,j_r=[],0
            print(f"lr_br:{optimizer.param_groups[0]['lr']}".upper())
            scheduler.step()
            print(f"lr_up:{optimizer.param_groups[0]['lr']}".upper())
            
    z_w_.append(err_real.item())
    if err_real.item()<high_rf:
            high_rf=err_real.item()
            torch.save(EFF_NET.state_dict(),f'/kaggle/working/jsr_rf/{err_real.item()}.pth')
    if i==10:
        break
    i+=1

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([100, 1])) that is different to the input size (torch.Size([200])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/opt/conda/lib/python3.10/site-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([75, 1])) that is different to the input size (torch.Size([150])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


EPOCH : 0 LOSS_wl : 107.97284698486328
EPOCH : 1 LOSS_wl : 107.8017807006836
EPOCH : 2 LOSS_wl : 109.00402069091797
EPOCH : 3 LOSS_wl : 110.2920150756836
EPOCH : 4 LOSS_wl : 107.88866424560547
EPOCH : 5 LOSS_wl : 107.78695678710938
EPOCH : 6 LOSS_wl : 108.06266784667969
EPOCH : 7 LOSS_wl : 107.80850982666016
EPOCH : 8 LOSS_wl : 107.93213653564453
EPOCH : 9 LOSS_wl : 108.19915771484375
LR_BR:0.0001
LR_UP:8.6E-05
EPOCH : 10 LOSS_wl : 107.9473648071289
